In [1]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import warnings
warnings.filterwarnings('ignore')

### Pregunta 1

def developer( desarrollador : str ): Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora. Ejemplo de retorno:

In [2]:
#Leemos el archivo reviews.parquet en un DataFrame
df_EP1 = pq.read_table("../0-DATA/1desarrollador.parquet").to_pandas()
df_EP1.head(5)

,price,developer,item_id,año
0,9.99,Valve,70,2015
1,9.99,Valve,70,2011
2,9.99,Valve,70,2014
3,9.99,Valve,70,2013
4,9.99,Valve,70,2013


In [3]:
def developer(desarrollador: str, df_EP1: pd.DataFrame):
    devel = df_EP1.developer.unique()
    if desarrollador not in devel:
        return {"message": "Mi base de datos no tiene registros de ese desarrollador"}

    listaaños = df_EP1.año.unique() #almaceno en una variable los valores unicos del campo Año
    listadeveloper = df_EP1.developer.unique() #almaceno en una variable los valores unicos del campo developer
    
    lista = [] # creamos una lista para añadir las variables buscadas
    
    if desarrollador in listadeveloper: #creamos una condicional para ver si está dentro de nuestra lista developer
        for años in listaaños: #iteramos
            todo = len(df_EP1[(df_EP1.año == años) & (df_EP1.developer == desarrollador)])
            cantidaditems = len(df_EP1[(df_EP1.price == 0) & (df_EP1.año == años) & (df_EP1.developer == desarrollador)])
            if todo != 0 or cantidaditems != 0:
                porcentaje = f"{round(cantidaditems*100/todo,2)}%"
                lista.append([años,cantidaditems,porcentaje])
            else:
                pass
    
    df = pd.DataFrame(lista, columns=["Año", "Cantidad de Items", "Contenido Free"])# Crear un DataFrame a partir del diccionario
    df.columns = ["Año", "Cantidad de Items", "Contenido Free"]# Agregar la fila de encabezado
    df = df.sort_values(by='Año',ascending=False)
    return df

In [4]:
developer('Valve', df_EP1) # 'Valve', 'Trion Worlds', 'Cryptic Studios', 'Grey Havens',
                                   #'Three Rings', 'Daybreak Game Company', 'Reloaded Productions',
                                   #'Splash Damage', 'CCP', 'Nimbly Games', 'Team Chivalry',
                                   #'PopCap Games, Inc.'otras empresas

,Año,Cantidad de Items,Contenido Free
0,2015,676,24.9%
2,2014,1609,45.92%
3,2013,817,56.38%
4,2012,275,66.11%
1,2011,88,53.33%
5,2010,15,68.18%


### Pregunta 2

def userdata( User_id : str ): Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.

In [5]:
#Leemos el archivo reviews.parquet en un DataFrame
df_EP2 = pq.read_table("../0-DATA/2User_id.parquet").to_pandas()
df_EP2.head(5)

,price,item_id,recommend,user_id
0,9.99,70,1,EizanAratoFujimaki
1,9.99,70,1,GamerFag
2,9.99,70,1,76561198020928326
3,9.99,70,1,Bluegills
4,9.99,70,1,76561198071955492


In [6]:
def userdata(usuario: str, df_EP2: pd.DataFrame):

    user = df_EP2.user_id.unique()
    if usuario not in user:
        return {"message": "Mi base de datos no tiene registros de ese usuario"}
    
    lista = df_EP2.user_id.unique()
    dfusuario = df_EP2[df_EP2.user_id == usuario]
    maneyuser = f"{round(dfusuario.price.sum(), 2)} USD"
    recomend = f"{int(len(dfusuario[dfusuario.recommend == 1]) * 100 / len(dfusuario))}%"
    itemss = len(dfusuario)
    data = {"User": usuario, 
                "Dinero gastado": maneyuser, "% de recomendación": recomend, "cantidad de items": itemss}
    return data 

In [7]:
userdata('voaDs', df_EP2)#'EizanAratoFujimaki', 'GamerFag', '76561198020928326', 'Bluegills',
                            #'76561198071955492', 'digfernandes', '76561197970812298',
                            #'meeeedie', 'phrostb', 'KewlKatzz', 'TheRealMeatyMate',
                            #'Auditore70', 'voaDs', '76561198060319088', '76561198085571748' u otros usuarios

{'User': 'voaDs',
 'Dinero gastado': '74.93 USD',
 '% de recomendación': '100%',
 'cantidad de items': 7}

### Pregunta 3

def UserForGenre( genero : str ): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año de lanzamiento.

In [8]:
#Leemos el archivo reviews.parquet en un DataFrame
df_EP3 = pq.read_table("../0-DATA/3genero.parquet").to_pandas()
df_EP3.head(5)

,genres_clean,user_id,año,playtime_hours
0,Action,EizanAratoFujimaki,2015,23.25
1,Action,GamerFag,2011,9.83
2,Action,76561198020928326,2014,93.32
3,Action,Bluegills,2013,1.07
4,Action,76561198071955492,2013,53.92


In [9]:
def UserForGenre(genero: str):
    
    gender = df_EP3.genres_clean.unique()
    if genero not in gender:
        return {"message": "Mi base de datos no tiene registros de ese genero"}

    años = df_EP3.año.unique()
    usuario_total = None
    horas_totales_usuario = 0
    user_data = []

    for añi in años:
        # Filter data for the specific genre and year
        df_genero = df_EP3[(df_EP3.genres_clean == genero) & (df_EP3.año == añi)]

        # Find user with most playtime hours
        if not df_genero.empty:
            usuario = df_genero.groupby('user_id')["playtime_hours"].sum().idxmax()
            horasT = df_genero[df_genero.user_id == usuario]["playtime_hours"].sum()

            # Update total user if necessary
            if horasT > horas_totales_usuario:
                usuario_total = usuario
                horas_totales_usuario = horasT

            # Append data for this year
            user_data.append({"Año": añi, "Horas": horasT})
        else:
            # No data found for this year
            user_data.append({"Año": añi, "Horas": 0})

    respuesta = {
        "Usuario con más horas jugadas para Género " + genero: usuario_total,
        "Horas jugadas": user_data,
    }

    return respuesta

In [10]:
UserForGenre('Massively Multiplayer')#'Action', 'Indie', 'Racing', 'Strategy', 'Casual', 'Simulation',
        #'Adventure', 'RPG', 'Sports', 'Free to Play',
        #'Animation &amp; Modeling', 'Utilities', 'Massively Multiplayer',
        #'Design &amp; Illustration', 'Video Production'

{'Usuario con más horas jugadas para Género Massively Multiplayer': 'mutatedwombat',
 'Horas jugadas': [{'Año': 2015, 'Horas': 4530.95},
  {'Año': 2011, 'Horas': 0},
  {'Año': 2014, 'Horas': 275.98},
  {'Año': 2013, 'Horas': 0},
  {'Año': 2012, 'Horas': 0},
  {'Año': 2010, 'Horas': 0}]}

### Pregunnta 4

def best_developer_year( año : int ): Devuelve el top 3 de desarrolladores con juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos)

In [11]:
#Leemos el archivo reviews.parquet en un DataFrame
df_EP4 = pq.read_table("../0-DATA/4año.parquet").to_pandas()
df_EP4.head(5)

,developer,recommend,user_id,sentiment_analysis,año
0,Valve,1,EizanAratoFujimaki,1,2015
1,Valve,1,GamerFag,0,2011
2,Valve,1,76561198020928326,1,2014
3,Valve,1,Bluegills,2,2013
4,Valve,1,76561198071955492,2,2013


In [12]:
def best_developer_year(year: int, df_EP4: pd.DataFrame):
    rango_aceptado = range(2010, 2016)
    if year not in rango_aceptado:
        return {"message": "Mi base de datos solo tiene registros entre 2010 y 2015"}
    #year = int(year)
    recom = df_EP4[(df_EP4.recommend == 1) & (df_EP4.sentiment_analysis == 2) & (df_EP4.año == year)]
    developer_recommendations = recom.groupby("developer")["recommend"].sum()
    top_3_developers = developer_recommendations.nlargest(3)

    top_developers_list = []
    for rank, (developer, count) in enumerate(top_3_developers.items(), start=1):
        top_developers_list.append({"Puesto " + str(rank): developer})

    return top_developers_list


In [13]:
best_developer_year(2012, df_EP4) # 2010,2011,2012,2013,2014,2015

[{'Puesto 1': 'Valve'},
 {'Puesto 2': 'Facepunch Studios'},
 {'Puesto 3': 'Re-Logic'}]

### Pregunta 5

def developer_reviews_analysis( desarrolladora : str ): Según el desarrollador, se devuelve un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor positivo o negativo.

In [14]:
#Leemos el archivo reviews.parquet en un DataFrame
df_EP5 = pq.read_table("../0-DATA/5desarrolladora.parquet").to_pandas()
df_EP5.head(5)

,developer,sentiment_analysis
0,Valve,1
1,Valve,0
2,Valve,1
3,Valve,2
4,Valve,2


In [15]:

def developer_reviews_analysis(desarrolladora: str, df_EP5: pd.DataFrame):
    develo = df_EP1.developer.unique()
    if desarrolladora not in develo:
        return {"message": "Mi base de datos no tiene registros de ese desarrollador"}
    
    # Filtramos por desarrolladora
    df_filt_developer = df_EP5[(df_EP5['developer'] == desarrolladora) & (df_EP5['sentiment_analysis'] != 1)]
    # Verificamos que haya datos para la desarrolladora
    if not df_filt_developer.empty:
        # Contamos los sentimientos y mapeamos el número del sentimiento a su etiqueta correspondiente
        sentiment_counts = df_filt_developer['sentiment_analysis'].value_counts()
        sentiment_mapping = {0: 'Negative', 2: 'Positive'}
        sentiment_counts.index = sentiment_counts.index.map(sentiment_mapping)
        
        result = {desarrolladora: sentiment_counts.to_dict()}
    else:
        result = 'No cuento con los registros de esa empresa en mi base de datos'

    return result

In [16]:
developer_reviews_analysis('id Software',df_EP5) #'Valve', 'Outerlight Ltd.', 'GlyphX Games',
                            #'Introversion Software', 'Facepunch Studios',
                            #'Bugbear Entertainment', 'Funcom',
                            #'Firaxis Games,Feral Interactive (Mac)',
                            #'Crystal Dynamics,Feral Interactive (Mac)', 'CAPCOM Co., Ltd.',
                            #'id Software', 'Gray Matter Studios', '2K Boston,2K Australia',
                            #'Relic Entertainment', 'The Creative Assembly'

{'id Software': {'Positive': 40, 'Negative': 7}}